In [3]:
import requests as req
import json
import io
import datetime
import os
# from pymongo import MongoClient
# from mongoengine import *

with open('config.json', 'r') as read_file:
    client = json.load(read_file)

# db_client = MongoClient('localhost', 27017)

In [20]:
# functions
def scp_query(query: str = '', inst: str = '', city: str = '', country: str = '', start_year: int = 0, end_year: int = 0):
    inst_q, city_q, country_q = '', '', ''
    if inst:
        inst_q = ' AND '.join(inst.lower().split(' '))
    if city:
        city_q = ' AND '.join(city.lower().split(' '))
    if country:
        country_q = ' AND '.join(country.lower().split(' '))
    query = [
        (f'{query}' if query else ''),
        (f'AFFIL({inst_q})' if inst_q else ''),
        (f'AFFILCITY({city_q})' if city_q else ''),
        (f'AFFILCOUNTRY({country_q})' if country_q else ''),
        (f'(PUBYEAR > {start_year - 1})' if start_year else ''),
        (f'(PUBYEAR < {end_year + 1})' if end_year != 0 else ''),
    ]
    query = filter(lambda element: element, query)
    return ' AND '.join(query)

def scp_lookup(client:dict, query: str, cnt, start_cursor: int = 0, count_per_page: int = 25):
    base = 'https://api.elsevier.com/content/search/scopus'
    params = {
        'query': query,
        'apiKey': client['apikey'],
        'insttoken': client['insttoken'],
        'view': 'COMPLETE',
        'start': f'{start_cursor + (cnt * count_per_page)}',
        'count': f'{count_per_page}',
    }
    headers = {'Accept': 'application/json'}
    return req.get(url=base, params=params)

def file_name(inst: str, year: int, folder: str = '', cnt: int = 0, extension: str = '.txt'):
    cnt = str(cnt + 1).zfill(5)
    isnt = inst.replace(' ', '-')
    if extension[0] != '.':
        extension += '.'
    return os.path.join(folder, f'{inst}_y{year}_{cnt}_{datetime.datetime.now()}')

def scp_papers(client: dict, query: str, years: list):
    for year in range(years[0], years[1]):
        query = scp_query(query=query, start_year=year, end_year=year)
        res = scp_lookup(client=client, query=query, cnt=0)
        total_results = int(res.json()['search-results']['opensearch:totalResults'])

In [4]:
start_year = 2017
end_year = 2017
access_type = 'ACCESSTYPE(OA)' #open access
access_type = 'ACCESSTYPE(OTHER)'
affil_IDs = [
    'AF-ID("Sharif University of Technology" 60027666)',
    'AF-ID("University of Tehran" 60022927)',
    'AF-ID("Tarbiat Modares University" 60032053)',
    'AF-ID("Amirkabir University of Technology" 60007751)',
    'AF-ID("Shahid Beheshti University" 60032873)',
    'AF-ID("Ferdowsi University of Mashhad" 60001800)',
]
query = scp_query(query=affil_IDs[3], start_year=start_year, end_year=end_year)

res = scp_lookup(client=client, query=query, cnt=0)

# res.json()
# titles = [doc['dc:title'] for doc in res.json()['search-results']['entry']]
# for t in titles:
#     print(t)
#     print()

In [5]:
res.json()['search-results']['opensearch:totalResults']

'2424'

In [83]:
with io.open('salam.json', 'w', encoding='utf8') as f:
    f.write(res.text) 

In [84]:
with io.open('salam.json', 'r', encoding='utf8') as ff:
    data = json.load(ff)

In [46]:
#database
db = db_client['amirkabir']
posts = db.posts
post_data = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun, you guys',
    'author': 'Scott'
}
result = posts.insert_one(post_data)
print('One post: {0}'.format(result.inserted_id))

One post: 5cc4330b8e36a50c2ce49466


In [27]:
for i in range(2010,2020):
    print(i)

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [22]:
file_name(folder='',inst='amirkabir university of technology',cnt=35,year=2018)

'amirkabir university of technology_y2018_00036_2019-04-29 10:17:11.331431'